In [1]:
%matplotlib widget
from helper import *

<h1>Introduction to Dual Quaternions</h1>

Dual quaternons are a compact representation that offers useful analytic properties. They were introduced by Clifford in 1873 (quelle). They are dual numbers in which the real and dual parts are quaternions:

$$
    \underline{\xi} = \hat{q}_r + \epsilon \hat{q}_d
$$

with $\underline{\xi} \in \mathbb{H}$ and the dual unit $\epsilon^2 = 0$ and $\epsilon \neq 0$.<br>
Similarly to <i>Homogeneours Transformation Matrices</i>, which are part of the <i>Special Euclidean Group</i> $\mathcal{SE}(3)$, dual quaternions can be used to represent spatial transformations. Dual quaternions used for spatial transformations are called <strong>unit dual quaternions</strong>. Here, the real part of the dual quaternion $Re(\underline{\xi}) =  \underline{\xi}_r = \hat{q}_r$ is a unit quaternion, which represents the orientation and rotation of the dual quaternion transformation. The dual part $Du(\underline{\xi}) =  \underline{\xi}_d = \hat{q}_d$ is not required to satisfy the unit magnitude requirement, as it represents the translation of the transformation. 

In [3]:
real = Quaternion(1, 0, 0, 0)
dual = Quaternion(0, 1, 2, 0)

dual_quaternion = DualQuaternion(real, dual)
print(dual_quaternion)

DualQuaternion(Real: Quaternion(1.000, 0.000, 0.000, 0.000), Dual: Quaternion(0.000, 1.000, 2.000, 0.000))



Multiplication of dual quaternions follows the same rules as for dual numbers, but also respects the rules of the quaternion multiplication:

$$
\begin{align*}
\underline{\xi}^{(1)} \otimes \underline{\xi}^{(2)} &= (q_r^{(1)} + \epsilon q_d^{(1)}) \otimes (q_r^{(2)} + \epsilon q_d^{(2)}) \\
&= (q_r^{(1)} \otimes q_r^{(2)}) + \epsilon(q_r^{(1)} \otimes q_d^{(2)} + q_d^{(1)} \otimes q_r^{(2)}) + \epsilon^2(q_d^{(1)} \otimes q_d^{(2)})
\end{align*}
$$

then, with $ \epsilon^2 = 0 $:

$$
\begin{equation}
\underline{\xi}^{(1)} \otimes \underline{\xi}^{(2)} = (q_r^{(1)} \otimes q_r^{(2)}) + \epsilon(q_r^{(1)} \otimes q_d^{(2)} + q_d^{(1)} \otimes q_r^{(2)}) \tag{1}
\end{equation}
$$

In the context of the thesis, dual quaternions are written with an underline $\underline{\bullet}$, and both quaternion and dualquaternion multiplication can are denoted as $\otimes$. The detailed and robust implementation of the dual quaternion algebra can once again be found in the created python package <cite id="4u137"><a href="#zotero%7C16222978%2FAGXR4PGH">(Temminghoff, 2023)</a></cite>.<br>
Similary to homogeneous transformations, the dual quaternion mulitplication can be used to compute successive transforms.

$$
{}_a\underline{\xi}^{c} = {}_a\underline{\xi}^{b} \otimes {}_b\underline{\xi}^{c}
$$

with the inverse transformation is defined as the conjugate of the dual quaternion, which is the conjugation of both dual and real part. It is analog to the inverse transformation from $\mathcal{SE}(3)$:

$$
{}_b\underline{\xi}^{a} = {}_a\underline{\xi}^{b*} = \hat{q}_r^* + \epsilon \hat{q}_d^*
$$

This multiplication can be written as matrix multiplication via the quaternion hamilton operators.
For this we first need to define the dual quaternion, analog as is the case for the quaternions, as vector $\underline{\xi}_{[vec]} \in \mathbb{R}^8$.

$$
\underline{\xi}_{[vec]} = [q_{r,w}, q_{r,x}, q_{r,y}, q_{r,z}, q_{d,w}, q_{d,x}, q_{d,y}, q_{d,z}]^T
$$

To construct the multiplication matrices $[\underline{\xi}]_{L}$ and $[\underline{\xi}]_{R}$, with $[\underline{\xi}] \in \mathbb{R}^{8 \times 8}$ the quaternion hamilton operators are recalled, and arranged blockwise to represent the dual quaternion basic multiplication. Once again there is a right and left matrix representation:

$$
\begin{align*}
\underline{\xi} \otimes \underline{\eta} &= 
\begin{bmatrix}
([\underline{\xi}_r]_L) & 0_{4x4} \\
([\underline{\xi}_d]_L) & ([\underline{\xi}_r]_L)
\end{bmatrix}_L \underline{\eta}_{[vec]} \\
&= 
\begin{bmatrix}
([\underline{\eta}_r]_R) & 0_{4x4} \\
([\underline{\eta}_d]_R) & ([\underline{\eta}_r]_R)
\end{bmatrix}_R \underline{\xi}_{[vec]}
\end{align*}
$$

Even though it might be suggested that $q_d$ is a pure quaternion, as it represents the position of the spatial transformation $\vec{p} \in \mathbb{R}^3$, it is not neccessarily the case. The dimensionality of the dual part depends on the cartesian translation $\bar{t} = (0, \vec{t}) = (0, t_x, t_y, t_z)$, which is a pure quaternion and the real part, representing the orientation and rotation, which is a unit quaternion. Depending on the orientation of the frame, the dual part $\hat{q}_d$ can take any form. The basic transformation which adheres to the rule of first translation, then translation is written as follows:

$$
    \underline{\xi} = q_r + \epsilon \frac{1}{2} \bar{t} \otimes q_r
$$

Inversely, with this relation, the extraction of $\vec{t}$ from a dual quaternion is possible. This operation is often neccessary as the dual quaternion position $\hat{q}_d$ is not phyically meaningful in euclidean space, but is defined in the dual quaternion space $\mathbb{H}$. 

$$
    \vec{t} = Im(2 \hat{q}_d \otimes \hat{q}_r^*)
$$

The following interactive demo shows this concept closer: We define the orientation of the spatial transformation $\hat{q}$ via axis angle and set the position vector $\vec{t}$ via three sliders to set the cartesian position. From this the dual quaternion $\underline{\xi}$ is constructed and printed, so that the structure of the dual quaternion can be examined.


In [11]:
fig, ax = create_3d_plot(Quaternion(1,0,0,0))
dual_quaternion_display = create_textbox("Dual Quaternion")

angle_slider = create_slider("theta", 0, -2*np.pi, 2*np.pi)
azimuth_slider = create_slider("azimuth", 0, -2*np.pi, 2*np.pi)
elevation_slider = create_slider("elevation", 0, -np.pi, np.pi)

x_slider = create_slider("x", 0, -1, 1)
y_slider = create_slider("y", 0, -1, 1)
z_slider = create_slider("z", 0, -1, 1)

rotation_axis = create_quiver(ax, [0,0,0], [1,0,0], 1, 'k', "rotation axis")
x_axis, y_axis, z_axis = draw_frame(ax, [0,0,0], np.eye(3))

# Update function for the sliders
def update_plot(change):
    global rotation_axis, x_axis, y_axis, z_axis

    rotation_axis.remove()
    x_axis.remove()
    y_axis.remove()
    z_axis.remove()
    
    direction = spherical_coordinates(azimuth_slider.value, elevation_slider.value)
    
    # construct unit quaternion from axis angle
    q_r = Quaternion.fromAxisAngle(angle_slider.value, direction)
    
    # set position vector
    pos = [x_slider.value, y_slider.value, z_slider.value]
    
    dq = DualQuaternion.fromQuatPos(q_r, pos)
    
    # update displays
    dual_quaternion_display.value = str(dq)
    
    # update the drawn vectors and the arc
    rotation_axis = create_quiver(ax, dq.getPosition().flatten(), direction,1, 'grey', "rotation axis")
    x_axis, y_axis, z_axis = draw_frame(ax, dq.getPosition().flatten(), q_r.asRotationMatrix())
    
    fig.canvas.draw()
    fig.canvas.flush_events()

    
angle_slider.observe(update_plot, names = 'value')
azimuth_slider.observe(update_plot, names='value')
elevation_slider.observe(update_plot, names='value')
x_slider.observe(update_plot, names='value')
y_slider.observe(update_plot, names='value')
z_slider.observe(update_plot, names='value')

widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.VBox([dual_quaternion_display, angle_slider, azimuth_slider, elevation_slider, x_slider, y_slider, z_slider]),
    pane_heights=[0, 2, 1]
)

AppLayout(children=(VBox(children=(Text(value='', description='Dual Quaternion', layout=Layout(width='98%')), …

<h3>Pure Dual Quaternions and the Line Transformation</h3>

Any arbitrary real vector $\vec{x} \in \mathbb{R}^6$ can be expressed as pure dual quaternion $\bar{\underline{\zeta}} = (0 + x_1i + x_2k + x_3j) + \epsilon(0 + x_4i + x_5k + x_6j)$, which can be transformed by any dual-quaternion $\underline{\xi}$.

$$ 
{}_b\bar{\underline{\zeta}} = {}_a\underline{\xi}^b \otimes {}_a\bar{\underline{\zeta}} \otimes {}_a\underline{\xi}^{b*} 
$$

This transformation formula is particularly powerful in robotics, where it can be used to manipulate and interpret the position and orientation of robotic elements in a three-dimensional space efficiently. It can be leveraged in areas where angular and linear velocities and accelerations have to be computed or transformed into different frames, such as in the <i>Recursive Newton Euler Algorithm</i>. In the classic algorithm both rotational and translational components have to be computed seperately. With the dual quaternion representation of the dual velocity and acceleration, where the dual velocity and acceleration ecapsules both rotational and translational terms <cite id="7c5rg"><a href="#zotero%7C16222978%2FZZC2ARLA">(Miranda De Farias et al., 2019)</a></cite>.


<a id="test"></a>

<!-- BIBLIOGRAPHY START -->
<div class="csl-bib-body">
  <div class="csl-entry"><i id="zotero|16222978/ZZC2ARLA"></i>Miranda De Farias, C., Da Cruz Figueredo, L. F., &#38; Yoshiyuki Ishihara, J. (2019). Performance Study on dqRNEA – A Novel Dual Quaternion Based Recursive Newton-Euler Inverse Dynamics Algorithms. <i>2019 Third IEEE International Conference on Robotic Computing (IRC)</i>, 94–101. <a href="https://doi.org/10.1109/IRC.2019.00022">https://doi.org/10.1109/IRC.2019.00022</a></div>
  <div class="csl-entry"><i id="zotero|16222978/AGXR4PGH"></i>Temminghoff, J. (2023). <i>JTem/neura_dual_quaternions</i>. <a href="https://github.com/JTem/neura_dual_quaternions">https://github.com/JTem/neura_dual_quaternions</a></div>
</div>
<!-- BIBLIOGRAPHY END -->